In [1]:
import numpy as np
import pandas as pd

train = pd.read_csv('\\Titanic\\train.csv')
test = pd.read_csv('\\Titanic\\test.csv')
full_data = [train, test]

#SibSp and Parch
for dataset in full_data:
    dataset['FamilySize']=dataset['SibSp']+dataset['Parch']+1
print(train[['FamilySize','Survived']].groupby(['FamilySize'],as_index=False).mean())    

for dataset in full_data:
    dataset['isalone']=0
    dataset['isalone'][dataset['FamilySize']==1]=1
print(train[['isalone','Survived']].groupby(['isalone'],as_index=False).mean())        


   FamilySize  Survived
0           1  0.303538
1           2  0.552795
2           3  0.578431
3           4  0.724138
4           5  0.200000
5           6  0.136364
6           7  0.333333
7           8  0.000000
8          11  0.000000
   isalone  Survived
0        0  0.505650
1        1  0.303538


C:\Users\abhi\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [2]:
#Embarked
value=dataset["Embarked"].dropna().mode()

for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
print (train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean())

train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
print (train[['CategoricalFare', 'Survived']].groupby(['CategoricalFare'], as_index=False).mean())


  Embarked  Survived
0        C  0.553571
1        Q  0.389610
2        S  0.339009
  CategoricalFare  Survived
0       [0, 7.91]  0.197309
1  (7.91, 14.454]  0.303571
2    (14.454, 31]  0.454955
3   (31, 512.329]  0.581081


In [4]:
for dataset in full_data:
    dataset["Fare"]=dataset["Fare"].fillna(dataset["Fare"].median())
    
for dataset in full_data:
    dataset['Age'] = dataset['Age'].fillna(dataset.Age.mean())
    
for dataset in full_data:
    dataset['Child']=np.where(dataset['Age']>=18,0,1)    

for dataset in full_data:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
       
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()    

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


In [5]:
    for dataset in full_data:
        #mapping Sex
        dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

        #mappingTitle
        title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
        dataset['Title']=dataset['Title'].map(title_mapping)
        dataset['Title']=dataset['Title'].fillna(0)

        # Mapping Embarked
        dataset["Embarked"]=dataset["Embarked"].map({'S':0,'C':1,'Q':2}).astype(int)

        dataset.loc[ dataset['Fare'] <= 7.91, 'Fare']= 0
        dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
        dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
        dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
        dataset['Fare'] = dataset['Fare'].astype(int)



    
        

In [6]:
# Feature Selection
drop_elements = ['Name', 'Age','Ticket', 'Cabin', 'SibSp','Parch', 'FamilySize']
train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalFare','PassengerId'], axis = 1)

test  = test.drop(drop_elements, axis = 1)

print (train.head(10))

   Survived  Pclass  Sex  Fare  Embarked  isalone  Child  Title
0         0       3    1     0         0        0      0      1
1         1       1    0     3         1        0      0      3
2         1       3    0     1         0        1      0      2
3         1       1    0     3         0        0      0      3
4         0       3    1     1         0        1      0      1
5         0       3    1     1         2        1      0      1
6         0       1    1     3         0        1      0      1
7         0       3    1     2         0        0      1      4
8         1       3    0     1         0        0      0      3
9         1       2    0     2         1        0      1      3


In [7]:
X_train = train.drop("Survived", axis=1)
Y_train = train["Survived"]
X_test  = test.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 7), (891,), (418, 7))

In [8]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
logreg =SVC()
logreg.fit(X_train, Y_train)
Y_pred1 = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

83.159999999999997

In [9]:
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": Y_pred1
    })
submission.to_csv("C:\\Users\\abhi\\Downloads\\Titanic\\kaggle_1.csv", index=False)